In [1]:
import nltk
import random
import spacy
from spacy import displacy
from collections import Counter
import typing
from typing import List, Dict 
import tqdm
import json
import pandas as pd
import pickle
import string 
import re

In [2]:
# download the dataset

nltk.download('reuters')
nltk.download('punkt')
from nltk.corpus import reuters

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\aubin\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aubin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# There are 4 main types of articles. The first 3 mentioned above do not hold any information about relation but simply about revenues, both quarterly and yearly.
# For a prelimnary analysis we will be ignoring these. One easy way of ignoring these is by excluding all articles that contain the word "qtr", the other one is where 
# "vs" is mentioned at least twice.

In [4]:
company_tickers = []

files = reuters.fileids()

suitable_articles = []
suitable_ids = []

for article_id in files:
    if reuters.raw(article_id).count("vs") <= 1:
        suitable_articles.append(reuters.raw(article_id))
        suitable_ids.append(article_id)

In [5]:
nlp = spacy.load('en_core_web_lg')

In [44]:
# data cleaning functionality
def clean_articles(sentence):

    tbc_sentence = sentence

    for i in range(tbc_sentence.count("&lt")):
        try:
            start = tbc_sentence.index("&lt")
            end = tbc_sentence.index(">")

            result = tbc_sentence[start + 4:end]
            company_tickers.append(result)

            tbc_sentence = tbc_sentence.replace(tbc_sentence[start:end+1], '')
        except:
            pass

    txt_no_return = tbc_sentence.replace("\n", " ") # remove "/n"
    txt_lowered = txt_no_return.lower()
    txt_no_lt = txt_lowered.replace('&lt', ' ')
    #text_without_punct = txt_no_lt.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    text_one_space = re.sub(' +', ' ', txt_no_lt) # remove multiple spaces
    text_clean = text_one_space.strip()
    return text_clean

In [45]:
articles_cleaned = {}

for idx, article in enumerate(suitable_articles):
    articles_cleaned[suitable_ids[idx]] = clean_articles(article)

In [46]:
filehandler = open("../stored_data/articles_cleaned.obj","wb")
pickle.dump(articles_cleaned, filehandler)
filehandler.close()

In [52]:
filehandler = open("../stored_data/company_tickers.obj","wb")
pickle.dump(company_tickers, filehandler)
filehandler.close()